In [1]:
# -*- coding: utf-8 -*-
"""
Preparación de datos para análisis espacial de género y 
\Created on Wed Aug 10 11:11:54 2022

@author: rafle
"""
"""
Referencia a datos de entrada
    1. Indice de pobreza multidimensional
    https://geoportal.dane.gov.co/visipm/
    src/pobrezamulti/VULNRB_IPMxMZ.shp

    2. Marco geoestadistico Nacional
    https://geoportal.dane.gov.co/servicios/descarga-y-metadatos/descarga-mgn-marco-geoestadistico-nacional/
    
    2.1. Manzanas censal
    src/mzn/MGN_URB_MANZANA.shp
    2.2. 
    3. Censo DANE 2018
    https://microdatos.dane.gov.co/index.php/catalog/643/get_microdata
   
    3.1. Microdatos  (TBD)
    
    3.2. Microdatos Atlantico
    3.2.1. Viviendas
    src/08Atlantico/
    3.2.2. Hogares
    src/08Atlantico/
    3.2.3. Fallecidos
    src/08Atlantico/
    3.2.4. Personas
    src/08Atlantico/
    3.2.5. Georreferenciación
    src/CENSO/08Atlantico/
"""

import pandas as pd
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np


In [2]:
import pandas as pd
geoRef = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_MGN_A2_08.csv')

In [3]:
## 00. Abrir información

## Abrir bases planas

'''
BARRANQUILLA
geoRef = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_MGN_A2_08.csv')
viviendas = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_1VIV_A2_08.csv')
hogares = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_2HOG_A2_08.csv')
fallecidos = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_3FALL_A2_08.csv')
personas = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_5PER_A2_08.csv')
'''

'''
VALLEDUPAR
geoRef = pd.read_csv('src/CENSO/20Cesar/CNPV2018_MGN_A2_20.csv')
viviendas = pd.read_csv('src/CENSO/20Cesar/CNPV2018_1VIV_A2_20.csv')
hogares = pd.read_csv('src/CENSO/20Cesar/CNPV2018_2HOG_A2_20.csv')
fallecidos = pd.read_csv('src/CENSO/20Cesar/CNPV2018_3FALL_A2_20.csv')
personas = pd.read_csv('src/CENSO/20Cesar/CNPV2018_5PER_A2_20.csv')
'''


geoRef = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_MGN_A2_08.csv')
viviendas = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_1VIV_A2_08.csv')
hogares = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_2HOG_A2_08.csv')
fallecidos = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_3FALL_A2_08.csv')
personas = pd.read_csv('src/CENSO/08Atlantico/CNPV2018_5PER_A2_08.csv')

# Abrir bases geográficas
geoManzanas = gpd.read_file('src/MGN/MZN/MGN_URB_MANZANA.shp')
municipios = gpd.read_file('src/MGN/PAIS/ADMINISTRATIVO/MGN_MPIO_POLITICO.shp')



In [6]:
## Dinamizar para diferentes areas de estudio
## Filtrar bases de datos
'''
FILTRO BAQ
municipiosFiltro = ['BARRANQUILLA','SOLEDAD']
FILTRO VDP
municipiosFiltro = ['VALLEDUPAR']
'''
municipiosFiltro = ['BARRANQUILLA','SOLEDAD']
municipiosAreaEstudio = municipios[municipios.MPIO_CNMBR.isin(municipiosFiltro)]

## Sjoin and filter, se puede reemplazar con un inner?
geoManzanasSJoin = gpd.sjoin(geoManzanas,municipiosAreaEstudio, 
                            how='left', op = 'intersects')
'''
geoManzanasMpio = geoManzanasSJoin[geoManzanasSJoin['DPTO_CNMBR']== 'CESAR']
'''
geoManzanasMpio = geoManzanasSJoin[geoManzanasSJoin['DPTO_CNMBR']== 'ATLÁNTICO']

## Simplificar dataset

geoManzanasSimp = geoManzanasMpio[['COD_DANE','DPTO_CNMBR',
                                  'MPIO_CNMBR','geometry']]
## Prueba de control

geoManzanasSimp.to_file('outputs/geomanzanasAreaEstudio.geojson',driver="GeoJSON")
geoManzanasSimp.to_file('outputs/geomanzanasAreaEstudio.shp')

C:\Users\Rafael.Munoz\Anaconda3\envs\Geoprocessing\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [7]:
## Filtrar datos de la encuesta departamental
geoRefAE = pd.merge(geoRef, geoManzanasSimp,
                   how ='inner', left_on='COD_DANE_ANM', right_on='COD_DANE')

In [8]:
## Personas
## 01. Agregar manzana

persManz = pd.merge(personas, geoRefAE[['MPIO_CNMBR','COD_ENCUESTAS','COD_DANE','geometry']], 
                    how ='inner', left_on = 'COD_ENCUESTAS', right_on = 'COD_ENCUESTAS')

## 00 Total de personas por manzana
totPersManz = (persManz.groupby('COD_DANE').count())[['U_DPTO']]
totPersManz.rename(columns = {'U_DPTO':'Tot_Pob'}, inplace = True)

## 01 Población por sexo
totPerHom = ((persManz[persManz['P_SEXO']== 1]).groupby('COD_DANE').count())[['U_DPTO']]
totPerHom.rename(columns= {'U_DPTO':'Tot_Hom'}, inplace = True)

totPerMuj = ((persManz[persManz['P_SEXO']== 2]).groupby('COD_DANE').count())[['U_DPTO']]
totPerMuj.rename(columns= {'U_DPTO':'Tot_Muj'}, inplace = True)

## 02 Edad

ninos = [1,2]
adole = [3,4]
joven = [5,6]
adult = [7,8,9,10,11,12]
mayor = [13,14,15,16,17,18,19,20,21]

def calcularPob(sexo, persManzana, grupoPob, nombre_cat):
    cenPob = persManzana[(persManzana['P_EDADR'].isin(grupoPob)) &
                        (persManzana['P_SEXO'] == sexo)]
    totPob = (cenPob.groupby('COD_DANE').count())[['U_DPTO']]
    totPob.rename(columns = {'U_DPTO': nombre_cat}, inplace = True)
    return totPob

pobNin_Hom = calcularPob(1, persManz, ninos, 'Tot_ninHom')
pobAdo_Hom = calcularPob(1, persManz, adole, 'Tot_adoHom')
pobJov_Hom = calcularPob(1, persManz, joven, 'Tot_jovHom')
pobAdu_Hom = calcularPob(1, persManz, adult, 'Tot_aduHom')
pobMay_Hom = calcularPob(1, persManz, mayor, 'Tot_mayHom')

pobNin_Muj = calcularPob(2, persManz, ninos, 'Tot_ninMuj')
pobAdo_Muj = calcularPob(2, persManz, adole, 'Tot_adoMuj')
pobJov_Muj = calcularPob(2, persManz, joven, 'Tot_jovMuj')
pobAdu_Muj = calcularPob(2, persManz, adult, 'Tot_aduMuj')
pobMay_Muj = calcularPob(2, persManz, mayor, 'Tot_mayMuj')


## 03 Grupo étnico

gruposEtnic = [1,2,3,4,5]

pobEtniHom = (persManz[(persManz.PA1_GRP_ETNIC.isin(gruposEtnic)) &
                        (persManz['P_SEXO'] == 1)].groupby('COD_DANE').count())[['U_DPTO']]
pobEtniHom.rename(columns = {'U_DPTO':'Pob_minH'}, inplace = True)

pobEtniMuj = (persManz[(persManz.PA1_GRP_ETNIC.isin(gruposEtnic)) &
                        (persManz['P_SEXO'] == 2)].groupby('COD_DANE').count())[['U_DPTO']]
pobEtniMuj.rename(columns = {'U_DPTO':'Pob_minM'}, inplace = True)

## 04 Migración hace 1 año

pobMigAnoHom = (persManz[(persManz['PA_VIVIA_1ANO'] == 4)&
                        (persManz['P_SEXO'] == 1)].groupby('COD_DANE').count())[['U_DPTO']]
pobMigAnoHom.rename(columns = {'U_DPTO':'Pob_migHom'}, inplace = True)

pobMigAnoMuj = (persManz[(persManz['PA_VIVIA_1ANO'] == 4)&
                        (persManz['P_SEXO'] == 2)].groupby('COD_DANE').count())[['U_DPTO']]
pobMigAnoMuj.rename(columns = {'U_DPTO':'Pob_migMuj'}, inplace = True)

## 05 Condición física

pobDiscapacHom = (persManz[(persManz['CONDICION_FISICA'] == 1) &
                        (persManz['P_SEXO'] == 1)].groupby('COD_DANE').count())[['U_DPTO']]
pobDiscapacHom.rename(columns = {'U_DPTO': 'Pob_disHom'}, inplace = True)

pobDiscapacMuj = (persManz[(persManz['CONDICION_FISICA'] == 1) &
                        (persManz['P_SEXO'] == 2)].groupby('COD_DANE').count())[['U_DPTO']]
pobDiscapacMuj.rename(columns = {'U_DPTO': 'Pob_disMuj'}, inplace = True)

## 06 Analfabetismo

pobAnalfHom = (persManz[(persManz['P_ALFABETA'] == 2) &
                        (persManz['P_EDADR'] >= 4)&
                        (persManz['P_SEXO'] == 1)].groupby('COD_DANE').count())[['U_DPTO']]
pobAnalfHom.rename(columns = {'U_DPTO': 'Pob_anaHom'}, inplace = True) 

pobAnalfMuj = (persManz[(persManz['P_ALFABETA'] == 2) &
                        (persManz['P_EDADR'] >= 4)&
                        (persManz['P_SEXO'] == 2)].groupby('COD_DANE').count())[['U_DPTO']]
pobAnalfMuj.rename(columns = {'U_DPTO': 'Pob_anaMuj'}, inplace = True) 

## 07 Nivel educativo

pobMayor15anos = (persManz[persManz['P_EDADR']>= 4])

## Grupos
ninguno = [10]
basica = [1,2,3,4]
mediaUnivers = [5,6,7]
posgrado= [8,9]

def calcularPobNivelEd(sexo, base, categorias,valores):
    poblacion = (base[(base.P_NIVEL_ANOSR.isin(categorias))&
                     (base['P_SEXO'] == sexo)].groupby('COD_DANE').count())[['U_DPTO']]
    poblacion.rename(columns = {'U_DPTO': valores},inplace = True)
    return poblacion

pob_noEduc_Hom = calcularPobNivelEd(1,pobMayor15anos, ninguno, 'pob_0NoHom')
pob_basica_Hom = calcularPobNivelEd(1,pobMayor15anos, basica,  'pob_1BaHom')
pob_univer_Hom = calcularPobNivelEd(1,pobMayor15anos, mediaUnivers, 'pob_2MeHom')
pob_posgra_Hom = calcularPobNivelEd(1,pobMayor15anos, posgrado, 'pob_3UnHom')

pob_noEduc_Muj = calcularPobNivelEd(2,pobMayor15anos, ninguno, 'pob_0NoMuj')
pob_basica_Muj = calcularPobNivelEd(2,pobMayor15anos, basica, 'pob_1BaMuj')
pob_univer_Muj = calcularPobNivelEd(2,pobMayor15anos, mediaUnivers, 'pob_2MeMuj')
pob_posgra_Muj = calcularPobNivelEd(2,pobMayor15anos, posgrado, 'pob_3UnMuj')

## 08 Trabajo

## Mayores a 20 años sin trabajos
sinTrabajo = [4]

pobNoTrabajo_Hom = (persManz[(persManz.P_TRABAJO.isin(sinTrabajo)) &
                            (persManz['P_SEXO'] ==1)&
                            (persManz['P_EDADR'] >=5 )].groupby('COD_DANE').count())[['U_DPTO']]

pobNoTrabajo_Hom.rename(columns = {'U_DPTO': 'Pob_NoTraH'}, inplace = True)

pobNoTrabajo_Muj = (persManz[(persManz.P_TRABAJO.isin(sinTrabajo)) &
                            (persManz['P_SEXO'] ==2) &
                            (persManz['P_EDADR'] >= 5)].groupby('COD_DANE').count())[['U_DPTO']]

pobNoTrabajo_Muj.rename(columns = {'U_DPTO': 'Pob_NoTraM'}, inplace = True)

## 09 Estado civil de las mujeres
## estado civil de mujeres mayores de 20 años

casada = [1,2]
viuda =  [6]
separada = [3,4,5]
solos = [3, 4, 5, 6, 7]

muj_CasadasUL = (persManz[(persManz['P_EDADR']>=4)&
                         (persManz['P_SEXO'] ==2)&
                         (persManz.P_EST_CIVIL.isin(casada))].groupby('COD_DANE').count())[['U_DPTO']]
muj_CasadasUL.rename(columns = {'U_DPTO': 'Muj_Casada'}, inplace = True)

muj_Separadas =(persManz[(persManz['P_EDADR']>=4)&
                         (persManz['P_SEXO'] ==2)&
                         (persManz.P_EST_CIVIL.isin(separada))].groupby('COD_DANE').count())[['U_DPTO']]
muj_Separadas.rename(columns = {'U_DPTO': 'Muj_Separa'}, inplace = True)

muj_Viudas = (persManz[(persManz['P_EDADR']>=4)&
                         (persManz['P_SEXO'] ==2)&
                         (persManz.P_EST_CIVIL.isin(viuda))].groupby('COD_DANE').count())[['U_DPTO']]
muj_Viudas.rename(columns = {'U_DPTO': 'Muj_Viudas'}, inplace = True)

## 10 Jefes de hogar

jef_Hogar_Hom = (persManz[(persManz['P_PARENTESCOR'] == 1) &
                          (persManz['P_SEXO'] == 1)].groupby('COD_DANE').count())[['U_DPTO']]
jef_Hogar_Hom.rename(columns = {'U_DPTO': 'Jefes_hoH'}, inplace = True)

jef_Hogar_Muj = (persManz[(persManz['P_PARENTESCOR'] == 1) &
                          (persManz['P_SEXO'] == 2)].groupby('COD_DANE').count())[['U_DPTO']]

jef_Hogar_Muj.rename(columns = {'U_DPTO': 'Jefes_hoM'}, inplace = True)

## 11 Hogares monoparentales

jef_hogar_Hom_Sol = (persManz[(persManz['P_PARENTESCOR']== 1) &
                              (persManz['P_SEXO']== 1) &
                              (persManz.P_EST_CIVIL.isin(solos))].groupby('COD_DANE').count())[['U_DPTO']]

jef_hogar_Hom_Sol.rename(columns = {'U_DPTO':'Hom_JH_Mo'}, inplace = True)

jef_hogar_Muj_Sol = (persManz[(persManz['P_PARENTESCOR']== 1) &
                              (persManz['P_SEXO']== 2) &
                              (persManz.P_EST_CIVIL.isin(solos))].groupby('COD_DANE').count())[['U_DPTO']]
jef_hogar_Muj_Sol.rename(columns = {'U_DPTO':'Muj_JH_Mo'}, inplace = True)


## Compilar en lista

resultadosCenso = [
    totPersManz,totPerHom, totPerMuj,
    pobNin_Hom, pobAdo_Hom, pobJov_Hom, pobAdu_Hom, pobMay_Hom,
    pobNin_Muj, pobAdo_Muj, pobJov_Muj, pobAdu_Muj, pobMay_Muj,
    pobEtniHom, pobEtniMuj,
    pobMigAnoHom, pobMigAnoMuj,
    pobDiscapacHom, pobDiscapacMuj,
    pobAnalfHom, pobAnalfMuj,
    pob_noEduc_Hom, pob_basica_Hom, pob_univer_Hom, pob_posgra_Hom,
    pob_noEduc_Muj, pob_basica_Muj, pob_univer_Muj, pob_posgra_Muj,
    pobNoTrabajo_Hom, pobNoTrabajo_Muj,
    muj_CasadasUL, muj_Separadas, muj_Viudas,
    jef_Hogar_Hom, jef_Hogar_Muj,
    jef_hogar_Hom_Sol, jef_hogar_Muj_Sol
]


In [9]:
### 
geoManzanasSimp
i = 1
contenedor = []
for variable in resultadosCenso:
    if i == 1:
        contenedor = pd.merge(geoManzanasSimp, variable, how='left',on='COD_DANE')
    else:
        contenedor = pd.merge(contenedor, variable,how='left',on='COD_DANE' )
    i = i+1
contenedor.fillna(0, inplace = True)
contenedor.to_file('outputs/01_Datos_nivel_personas.shp')

In [10]:
contenedor

,COD_DANE,DPTO_CNMBR,MPIO_CNMBR,geometry,Tot_Pob,Tot_Hom,Tot_Muj,Tot_ninHom,Tot_adoHom,Tot_jovHom,...,pob_3UnMuj,Pob_NoTraH,Pob_NoTraM,Muj_Casada,Muj_Separa,Muj_Viudas,Jefes_hoH,Jefes_hoM,Hom_JH_Mo,Muj_JH_Mo
0,0800110000000000010101,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.87138 11.03406, -74.87147 11.034...",74.0,35.0,39.0,6.0,12.0,6.0,...,0.0,1.0,0.0,14.0,4.0,2.0,9.0,11.0,1.0,5.0
1,0800110000000000010102,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.86919 11.03445, -74.86919 11.034...",161.0,82.0,79.0,15.0,17.0,20.0,...,1.0,2.0,3.0,31.0,8.0,0.0,23.0,16.0,4.0,7.0
2,0800110000000000010103,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.86885 11.03357, -74.86917 11.033...",164.0,79.0,85.0,20.0,15.0,10.0,...,7.0,7.0,0.0,37.0,0.0,1.0,15.0,21.0,5.0,4.0
3,0800110000000000010105,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.86695 11.03367, -74.86686 11.033...",425.0,217.0,208.0,39.0,38.0,36.0,...,5.0,4.0,7.0,89.0,12.0,4.0,76.0,35.0,16.0,17.0
4,0800110000000000010106,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.86466 11.03510, -74.86466 11.035...",414.0,207.0,207.0,31.0,37.0,43.0,...,12.0,5.0,11.0,96.0,17.0,4.0,83.0,24.0,12.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15446,0875810000000000420111,ATLÁNTICO,SOLEDAD,"POLYGON ((-74.80973 10.90694, -74.80956 10.906...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15447,0875810000000000420112,ATLÁNTICO,SOLEDAD,"POLYGON ((-74.81027 10.90667, -74.81001 10.907...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15448,0875810000000000420113,ATLÁNTICO,SOLEDAD,"POLYGON ((-74.81008 10.90712, -74.81020 10.906...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15449,0875810000000000420114,ATLÁNTICO,SOLEDAD,"POLYGON ((-74.81236 10.90680, -74.81225 10.907...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
## Viviendas

vivManz = pd.merge(viviendas, geoRefAE[['MPIO_CNMBR','COD_ENCUESTAS','COD_DANE','geometry']],
                   how = 'inner',
                   left_on = 'COD_ENCUESTAS',
                   right_on = 'COD_ENCUESTAS')

## Contar viviendas por manzana
totVivManz =(vivManz.groupby('COD_DANE').count())[['U_DPTO']]
totVivManz.reset_index(inplace = True)

## Funcion agrupar viviendas en manzanas
def contarViv(dataset, variable, valores):
    noElemento = dataset[dataset[variable].isin(valores)]
    totNoElemento= (noElemento.groupby('COD_DANE').count())[['U_DPTO']]
    totNoElemento.reset_index(inplace = True)
    return totNoElemento

## Filtro energía
totVivNoEner = contarViv(vivManz, 'VA_EE', [2])

## Filtro Acueducto
totVivNoAcue = contarViv(vivManz, 'VB_ACU', [2])

## Filtro Alcantarillado
totVivNoAlca = contarViv(vivManz, 'VC_ALC', [2])

## Filtro Baño
totVivNoBano = contarViv(vivManz, 'V_TIPO_SERSA', [3,4,5,6])

## Pisos de tierra y otros no arreglados
totVivNoPiso = contarViv(vivManz, 'V_MAT_PISO', [5,6])

## Viviendas sin recolección de basuras

totVivNoBas = contarViv(vivManz, 'VE_RECBAS', [2] )

## Viviendas por estrato
totVivEstr0 = contarViv(vivManz, 'VA1_ESTRATO',[0])
totVivEstr1 = contarViv(vivManz, 'VA1_ESTRATO',[1])
totVivEstr2 = contarViv(vivManz, 'VA1_ESTRATO',[2])
totVivEstr3 = contarViv(vivManz, 'VA1_ESTRATO',[3])
totVivEstr4 = contarViv(vivManz, 'VA1_ESTRATO',[4])
totVivEstr5 = contarViv(vivManz, 'VA1_ESTRATO',[5])
totVivEstr6 = contarViv(vivManz, 'VA1_ESTRATO',[6])
totVivEstr9 = contarViv(vivManz, 'VA1_ESTRATO',[9])


In [12]:
totVivNoEner

,COD_DANE,U_DPTO
0,0800110000000000010110,1
1,0800110000000000010132,1
2,0800110000000000010134,2
3,0800110000000000010313,2
4,0800110000000000010314,2
...,...,...
839,0875810000000000410610,1
840,0875810000000000410612,2
841,0875810000000000410704,1
842,0875810000000000410705,2


In [13]:
## Juntar todas las bases

vivNoEner = pd.merge(totVivManz, totVivNoEner, 
                     how='left',
                     left_on = 'COD_DANE', 
                     right_on = 'COD_DANE')
vivNoEner.rename(columns = {'U_DPTO_x':'viv_Total','U_DPTO_y':'viv_NoEner'}, inplace = True)

vivNoAcue= pd.merge(vivNoEner, totVivNoAcue, how= 'left', on ='COD_DANE').rename(columns={'U_DPTO':'viv_NoAcue'})
vivNoAlca= pd.merge(vivNoAcue, totVivNoAlca, how= 'left', on ='COD_DANE').rename(columns={'U_DPTO':'viv_NoAlca'})
vivNoBano= pd.merge(vivNoAlca, totVivNoBano, how= 'left', on ='COD_DANE').rename(columns={'U_DPTO':'viv_NoBano'})
vivNoPiso= pd.merge(vivNoBano, totVivNoPiso, how= 'left', on ='COD_DANE').rename(columns={'U_DPTO':'viv_NoPiso'})
vivNoBasu= pd.merge(vivNoPiso, totVivNoBas,  how= 'left', on ='COD_DANE').rename(columns={'U_DPTO':'viv_NoRecB'})

vivEstra0= pd.merge(vivNoBasu, totVivEstr1, how='left', on='COD_DANE').rename(columns={'U_DPTO':'Viv_Estr_0'})
vivEstra1= pd.merge(vivEstra0, totVivEstr1, how='left', on='COD_DANE').rename(columns={'U_DPTO':'Viv_Estr_1'})
vivEstra2= pd.merge(vivEstra1, totVivEstr2, how='left', on='COD_DANE').rename(columns={'U_DPTO':'Viv_Estr_2'})
vivEstra3= pd.merge(vivEstra2, totVivEstr3, how='left', on='COD_DANE').rename(columns={'U_DPTO':'Viv_Estr_3'})
vivEstra4= pd.merge(vivEstra3, totVivEstr4, how='left', on='COD_DANE').rename(columns={'U_DPTO':'Viv_Estr_4'})
vivEstra5= pd.merge(vivEstra4, totVivEstr5, how='left', on='COD_DANE').rename(columns={'U_DPTO':'Viv_Estr_5'})
vivEstra6= pd.merge(vivEstra5, totVivEstr6, how='left', on='COD_DANE').rename(columns={'U_DPTO':'Viv_Estr_6'})
vivEstra9= pd.merge(vivEstra6, totVivEstr9, how='left', on='COD_DANE').rename(columns={'U_DPTO':'Viv_Estr_9'})


vivGeogra= pd.merge(geoManzanasSimp, vivEstra9, how='left', on ='COD_DANE')
vivGeogra.fillna(0, inplace = True)

In [14]:
vivGeogra
vivGeogra.to_file('outputs/02_Datos_Nivel_Viviendas.geojson',driver="GeoJSON")
vivGeogra.to_file('outputs/02_Datos_Nivel_Viviendas.shp')


In [15]:
vivGeogra

,COD_DANE,DPTO_CNMBR,MPIO_CNMBR,geometry,viv_Total,viv_NoEner,viv_NoAcue,viv_NoAlca,viv_NoBano,viv_NoPiso,viv_NoRecB,Viv_Estr_0,Viv_Estr_1,Viv_Estr_2,Viv_Estr_3,Viv_Estr_4,Viv_Estr_5,Viv_Estr_6,Viv_Estr_9
0,0800110000000000010101,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.87138 11.03406, -74.87147 11.034...",21.0,0.0,0.0,5.0,0.0,1.0,1.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0800110000000000010102,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.86919 11.03445, -74.86919 11.034...",42.0,0.0,0.0,14.0,1.0,11.0,2.0,39.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0800110000000000010103,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.86885 11.03357, -74.86917 11.033...",36.0,0.0,0.0,0.0,4.0,16.0,0.0,20.0,20.0,16.0,0.0,0.0,0.0,0.0,0.0
3,0800110000000000010105,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.86695 11.03367, -74.86686 11.033...",114.0,0.0,2.0,3.0,2.0,5.0,0.0,109.0,109.0,2.0,0.0,0.0,0.0,0.0,0.0
4,0800110000000000010106,ATLÁNTICO,BARRANQUILLA,"POLYGON ((-74.86466 11.03510, -74.86466 11.035...",111.0,0.0,1.0,16.0,0.0,2.0,0.0,99.0,99.0,5.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15446,0875810000000000420111,ATLÁNTICO,SOLEDAD,"POLYGON ((-74.80973 10.90694, -74.80956 10.906...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15447,0875810000000000420112,ATLÁNTICO,SOLEDAD,"POLYGON ((-74.81027 10.90667, -74.81001 10.907...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15448,0875810000000000420113,ATLÁNTICO,SOLEDAD,"POLYGON ((-74.81008 10.90712, -74.81020 10.906...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15449,0875810000000000420114,ATLÁNTICO,SOLEDAD,"POLYGON ((-74.81236 10.90680, -74.81225 10.907...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
test = vivManz[vivManz['COD_DANE'] == '0800110000000033010311']


test.to_csv('test.csv')